In [43]:
import matplotlib.pyplot as plt
import gensim
from gensim.corpora import Dictionary
import numpy as np
import spacy
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import nltk
nltk.download('wordnet')
import pyLDAvis.gensim
import re


import os, re, operator, warnings
warnings.filterwarnings('ignore')  # Let's not pay heed to them right now
%matplotlib inline

n_samples = 100 
n_features = 1000
n_topics = 10


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rahuljain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
dataset = fetch_20newsgroups( shuffle=True, random_state=1, remove=('headers','footers','quotes'))
data_samples = dataset.data[:n_samples]

In [33]:
len(data_samples[1])

546

In [16]:
##tf_vectorizer = CountVectorizer(stop_words=None, max_df=0.95,min_df=2, max_features=n_features)
##tf = tf_vectorizer.fit_transform(data_samples)

In [34]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop_words = set(stopwords.words('english'))
punc_words = set(string.punctuation)
lemma_words = WordNetLemmatizer().lemmatize

In [42]:
type(data_samples[1])

str

In [97]:
news_doc = []
for sent in data_samples:
    doc = re.findall(r'\b[a-b]{3,}\b',sent)
    news_doc.append(doc)

In [98]:
data_samples[0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [71]:
type(check) 

_sre.SRE_Match

In [35]:
##Input passed is a paragraph/doc and returns a clean paragraph/doc
def clean_text(para):
    stop_punc_free = ' '.join(word for word in para.split() if word not in stop_words and word not in punc_words )
    normalized     = ' '.join(lemma_words(word) for word in stop_punc_free.split())
    email_free     = re.sub(r'([a-zA-Z0-9]+\.)*[a-zA-Z0-9]+@([a-zA-Z0-9]+\.)*[a-zA-Z]+','',normalized)
    bar_free       = re.sub(r'(=*)','',email_free)
    bar_free2      = re.sub(r'(-*)','',bar_free)
    ip_free        = re.sub(r'\[(\d{1,3}\.)*\d{1,3}\]:*','',bar_free2)
    tag_free       = re.sub(r'<[^<>]*>','',ip_free)
    phone_free     = re.sub(r'(\(\d{1,3}\)\s?\d{1,7})|(\(?\d{1,10}\)?)|(#)|(\+)','',tag_free)
    return(phone_free)

In [36]:
##Cleaning the data and converting the data into Corpus - List of lists
clean_data = [clean_text(para).split() for para in data_samples]

In [37]:
clean_data

[['Well',
  "i'm",
  'sure',
  'story',
  'nad',
  'seem',
  'biased.',
  'What',
  'I',
  'disagree',
  'statement',
  'U.S.',
  'Media',
  'ruin',
  'Israels',
  'reputation.',
  'That',
  'rediculous.',
  'The',
  'U.S.',
  'medium',
  'proisraeli',
  'medium',
  'world.',
  'Having',
  'lived',
  'Europe',
  'I',
  'realize',
  'incidence',
  'one',
  'described',
  'letter',
  'occured.',
  'The',
  'U.S.',
  'medium',
  'whole',
  'seem',
  'try',
  'ignore',
  'them.',
  'The',
  'U.S.',
  'subsidizing',
  'Israels',
  'existance',
  'Europeans',
  '(at',
  'least',
  'degree).',
  'So',
  'I',
  'think',
  'might',
  'reason',
  'report',
  'clearly',
  'atrocities.',
  'What',
  'shame',
  'Austria,',
  'daily',
  'report',
  'inhuman',
  'act',
  'commited',
  'Israeli',
  'soldier',
  'blessing',
  'received',
  'Government',
  'make',
  'Holocaust',
  'guilt',
  'go',
  'away.',
  'After',
  'all,',
  'look',
  'Jews',
  'treating',
  'race',
  'got',
  'power.',
  'It',
  

In [12]:
##Creating a dictionary of all the words
dictionary = Dictionary(clean_data)

In [13]:
type(dictionary),len(dictionary)

(gensim.corpora.dictionary.Dictionary, 52431)

In [14]:
for x in dictionary.items():
    print(x)


(0, 'Well')
(1, "i'm")
(2, 'sure')
(3, 'story')
(4, 'nad')
(5, 'seem')
(6, 'biased.')
(7, 'What')
(8, 'I')
(9, 'disagree')
(10, 'statement')
(11, 'U.S.')
(12, 'Media')
(13, 'ruin')
(14, 'Israels')
(15, 'reputation.')
(16, 'That')
(17, 'rediculous.')
(18, 'The')
(19, 'medium')
(20, 'proisraeli')
(21, 'world.')
(22, 'Having')
(23, 'lived')
(24, 'Europe')
(25, 'realize')
(26, 'incidence')
(27, 'one')
(28, 'described')
(29, 'letter')
(30, 'occured.')
(31, 'whole')
(32, 'try')
(33, 'ignore')
(34, 'them.')
(35, 'subsidizing')
(36, 'existance')
(37, 'Europeans')
(38, '(at')
(39, 'least')
(40, 'degree).')
(41, 'So')
(42, 'think')
(43, 'might')
(44, 'reason')
(45, 'report')
(46, 'clearly')
(47, 'atrocities.')
(48, 'shame')
(49, 'Austria,')
(50, 'daily')
(51, 'inhuman')
(52, 'act')
(53, 'commited')
(54, 'Israeli')
(55, 'soldier')
(56, 'blessing')
(57, 'received')
(58, 'Government')
(59, 'make')
(60, 'Holocaust')
(61, 'guilt')
(62, 'go')
(63, 'away.')
(64, 'After')
(65, 'all,')
(66, 'look')
(67, 

(624, '"help"')
(625, '(without')
(626, 'quotes!)')
(627, 'body.')
(628, 'place')
(629, 'Listing.')
(630, 'See')
(631, 'item:')
(632, 'Places')
(633, 'find')
(634, 'information.')
(635, 'Items')
(636, 'Changed:')
(637, 'REARRANGED')
(638, 'subjects,')
(639, 'order')
(640, 'fir')
(641, 'better')
(642, 'K/article')
(643, 'limit.')
(644, 'PLAN')
(645, 'ON')
(646, 'CHANGING')
(647, 'HEADERS')
(648, 'SOON,')
(649, 'SO')
(650, 'BE')
(651, 'CAREFUL!')
(652, 'ONLY')
(653, 'THE')
(654, '"Resource')
(655, 'Listing"')
(656, 'remain')
(657, 'Subject:')
(658, 'line!')
(659, 'D')
(660, 'graphic')
(661, 'editor')
(662, 'a.')
(663, 'Public')
(664, 'domain,')
(665, 'free')
(666, 'shareware')
(667, 'system')
(668, 'Plotting')
(669, 'package')
(670, 'Molecular')
(671, 'visualization')
(672, 'stuff')
(673, 'thinking')
(674, 'making')
(675, 'biweekly.')
(676, 'think???')
(677, 'Lines')
(678, 'changed,')
(679, "`'")
(680, 'character')
(681, 'front')
(682, 'Added')
(683, 'prepended')
(684, 'Removed')
(685, '

(2151, 'atoms,')
(2152, 'Atomic')
(2153, 'coordinate')
(2154, 'load')
(2155, 'stuff)')
(2156, 'contained')
(2157, '"*.ent"')
(2158, 'files,')
(2159, 'atomic')
(2160, 'dimemsions')
(2161, 'missing.')
(2162, 'PoV,')
(2163, 'rayshade,')
(2164, 'biome.bio.ns.ca')
(2165, '/pub/art')
(2166, 'Renoir')
(2167, 'paintings,')
(2168, "Escher's")
(2169, 'pictures,')
(2170, 'ic.ee.umanitoba.ca')
(2171, '[]')
(2172, '/specmark')
(2173, 'sample')
(2174, '`Images')
(2175, "Edge'")
(2176, 'CDROM')
(2177, '(images')
(2178, 'landscapes,')
(2179, 'advanced')
(2180, 'semiconductors,')
(2181, 'superconductors')
(2182, 'experimental')
(2183, 'chemistry')
(2184, 'among')
(2185, 'others).')
(2186, 'explorer.dgp.toronto.edu')
(2187, 'pub/sgi/clrpaint')
(2188, '*CLRpaint*')
(2189, 'pub/sgi/clrview.*')
(2190, 'CLRview,')
(2191, 'aid')
(2192, 'DXF,')
(2193, 'DEM,')
(2194, 'Arc/Info,')
(2195, 'ames.arc.nasa.gov')
(2196, 'pub/SPACE/CDROM')
(2197, 'Magellan')
(2198, 'Viking')
(2199, 'mission')
(2200, 'Get')
(2201, 'pu

(3507, 'Technical')
(3508, 'Services')
(3509, 'Standards,')
(3510, 'ext.')
(3511, 'PDES/STEP')
(3512, 'slowly')
(3513, 'emerging')
(3514, 'encompass')
(3515, 'FEM,')
(3516, 'main')
(3517, 'besides')
(3518, 'NIST')
(3519, 'DARPA.')
(3520, 'PDES')
(3521, 'putting')
(3522, '(NOT')
(3523, 'Subject:)')
(3524, 'Rutherford')
(3525, 'Appleton')
(3526, 'Lab.')
(3527, 'working')
(3528, 'STEP')
(3529, 'tools:')
(3530, 'EXPRESS')
(3531, 'compiler')
(3532, 'Exchange')
(3533, 'parser,')
(3534, 'form')
(3535, 'free)')
(3536, 'purposes.')
(3537, 'EXPRESSbased')
(3538, 'Mike')
(3539, 'Mead,')
(3540, '(FAX:')
(3541, '{...!}mcsun!uknet!rlinf!mm')
(3542, 'mm%relay.ac.uk')
(3543, 'End')
(3544, 'Part')
(3545, 'Roberto')
(3546, 'Clemente')
(3547, 'Topps')
(3548, 'baseball')
(3549, 'sale,')
(3550, 'nearmint')
(3551, 'condition')
(3552, '(really')
(3553, 'mint')
(3554, 'get).')
(3555, 'recent')
(3556, 'pricelist')
(3557, 'May.')
(3558, 'offering')
(3559, 'pay')
(3560, 'certified')
(3561, 'postage')
(3562, 'shi

(5131, 'conductors,')
(5132, 'magnet')
(5133, '(metal,')
(5134, 'ceramic),')
(5135, 'magnetostrictives,')
(5136, 'etc).')
(5137, 'NOTES:')
(5138, 'summarize,')
(5139, 'stated')
(5140, 'detectable')
(5141, 'vacuum.')
(5142, 'is,')
(5143, 'particle')
(5144, 'presence')
(5145, 'matter.')
(5146, 'Previous')
(5147, 'certain')
(5148, 'semiconductors.')
(5149, 'naturally')
(5150, 'arises')
(5151, 'vanish')
(5152, 'joined')
(5153, 'conductor')
(5154, '(say,')
(5155, 'copper).')
(5156, 'theoretical')
(5157, 'metal')
(5158, 'amplify')
(5159, 'detect')
(5160, '*if*')
(5161, 'sci.electronics')
(5162, 'becoming')
(5163, 'discussion.')
(5164, 'Thanx,')
(5165, 'Eric.')
(5166, 'ET')
(5167, '"A')
(5168, 'Force')
(5169, 'Nature"')
(5170, 'Hi,')
(5171, 'SIRD')
(5172, 'picture,')
(5173, 'stereogram')
(5174, 'eye')
(5175, 'becomes')
(5176, 'one?')
(5177, 'replies.')
(5178, 'thanks')
(5179, '.Chris')
(5180, '*Reminder*')
(5181, 'Plan')
(5182, 'Andrew')
(5183, 'Conference.')
(5184, '*Date*')
(5185, 'date')
(

...

(45093, 'spleen,')
(45094, "Matza's")
(45095, 'pompous')
(45096, 'laudable')
(45097, 'precaution.')
(45098, 'negatively')
(45099, 'Deliberate')
(45100, 'omission')
(45101, 'shtick.')
(45102, 'mention,')
(45103, 'fraught')
(45104, 'omissions,')
(45105, 'absurdity')
(45106, 'bias.')
(45107, 'unfavorable')
(45108, 'Paraphrase')
(45109, 'court?')
(45110, 'Fight')
(45111, 'ed.')
(45112, 'Berkeley,')
(45113, 'Nolo')
(45114, "UCB's")
(45115, 'library).')
(45116, 'luck;')
(45117, 'persecuted')
(45118, 'imprisoned')
(45119, 'pornography,')
(45120, 'capitalist,')
(45121, 'affairs,')
(45122, 'cryptography,')
(45123, 'drugs/guns/pornography,')
(45124, 'allowed!)')
(45125, 'Sell')
(45126, 'PIT')
(45127, 'Mario_Lemieux')
(45128, 'BUF')
(45129, 'Pat_LaFontaine')
(45130, 'BOS')
(45131, 'Adam_Oates')
(45132, 'DET')
(45133, 'Steve_Yzerman')
(45134, 'WPG')
(45135, 'Teemu_Selanne')
(45136, 'Pierre_Turgeon')
(45137, 'TOR')
(45138, 'Doug_Gilmour')
(45139, 'Alexander_Mogilny')
(45140, 'PHI')
(45141, 'Mark_Re

(46431, 'ESDI,')
(46432, '{This')
(46433, 'IDE..."')
(46434, '{this')
(46435, 'SCSI}')
(46436, 'CONSITANTLY')
(46437, 'read:')
(46438, '{asynchronous')
(46439, 'NONUSE')
(46440, 'eliminates')
(46441, 'ambaguity.')
(46442, 'expansive')
(46443, '{bit')
(46444, 'degridation(the')
(46445, 'EXACTLY')
(46446, 'this.)}')
(46447, '}>}>}(a)')
(46448, 'context;')
(46449, '}>}>Must')
(46450, '"promises')
(46451, '}>}>getting')
(46452, '}>}>Or')
(46453, 'hypocrisy.')
(46454, '}>}')
(46455, '}>}So,')
(46456, '}>}taking')
(46457, 'hypocrite,')
(46458, '}>}response')
(46459, 't.r.m')
(46460, 'context?')
(46461, '}>')
(46462, '}>Did')
(46463, 'misaimed')
(46464, 'telepathy')
(46465, 'A)')
(46466, 'creationist')
(46467, 'error)')
(46468, 'condoned')
(46469, 'Excellent.')
(46470, '"Could')
(46471, 'nonsequitur,')
(46472, '}Is')
(46473, 'Jim?')
(46474, '}explain')
(46475, '}may')
(46476, 'hasty.)')
(46477, '"yup."')
(46478, 'Quist')
(46479, 'bemoaned:')
(46480, 'buyout.')
(46481, 'pricey.')
(46482, 'Vick

(47972, 'now...)?')
(47973, 'appreciated...thanks!')
(47974, 'Headers')
(47975, 'uupsi')
(47976, 'aolsys.aol.com')
(47977, 'EXPO.LCS.MIT.EDU')
(47978, 'uu.psi.com')
(47979, '.b/..PSI/PSINet)')
(47980, 'SMTP;')
(47981, 'expo.lcs.mit.edu;')
(47982, 'ENTERPOOP.MIT.EDU')
(47983, 'enterpoop.MIT.EDU')
(47984, 'enterpoop')
(47985, 'netnewsfor')
(47986, '();contact')
(47987, 'Astronomy,')
(47988, 'XPutImage()?')
(47989, 'GLH')
(47990, '"hitthegroundrunning"')
(47991, 'technique.')
(47992, 'sand/gravel')
(47993, 'unkosher')
(47994, 'asofyet')
(47995, 'unidentified,')
(47996, 'intimately')
(47997, 'acquaint')
(47998, 'darkening')
(47999, 'behaving,')
(48000, 'gravitationally')
(48001, 'brick).')
(48002, 'Negotiating')
(48003, 'Daytona')
(48004, '(sans')
(48005, 'HO,')
(48006, 'sun,')
(48007, 'babes)')
(48008, 'destiny.')
(48009, "Lucifer's")
(48010, 'DOG.')
(48011, 'blend')
(48012, 'canine')
(48013, 'Doberman')
(48014, 'Rottweiler')
(48015, 'Doberman,')
(48016, 'hulk')
(48017, 'jowels')
(48018, 

(49610, 'M)P[?,QODZ,EG:P;HGVC>V?')
(49611, '[@V,$E*.PY')
(49612, 'GAR/(?C;\\["LSX')
(49613, 'M@?/CNOI"LUMX@;/[N!IV]S^F*I\'(\\B.,]UF%DA')
(49614, 'W(SRX!:W@/')
(49615, 'M:"F!"B.UZ"\\UG@!)SP:NY')
(49616, ']$,!YO_V]\\$:/DPNA:HR.!_/')
(49617, 'M!^N@B^][_CUVQUOGNP[FAYJK(__FMPIY[N%;SA.F')
(49618, "M)SB$/^Z:CUPI<*%T:J'BXE/UBXCM,;M;/KR?BNSZB/OYT/")
(49619, 'MA]*XF_T;/WA$&E"&E&:F,UMCD%/VU,\\\'LEN)FQC"\\F%)Z;')
(49620, 'MO].^/EF\'YH\'[/LVZWLTSDJ"R\\O/V<JO"B_,.')
(49621, '[VHS)[')
(49622, 'M!.<\'P<;%"N_GZX")UG\'CD]I^L%RKCL/TELTZHP]V_;K')
(49623, 'M_%S<"MF/Q')
(49624, "MOF?XP<P:X]PG'W@?^OWI]Z/?EU[@MQX\\?AHPDL',")
(49625, ',$&X)I^_ULJ,EJ!N<"G')
(49626, 'YT')
(49627, 'RRX,]\\')
(49628, "\\_CZXN'W[ST")
(49629, "?/WVO_K'AXK.G")
(49630, 'ME\\')
(49631, '>ISURL]RT[;NP')
(49632, '??N\'E%A?.>\'T"RUOV].OPO&,\'IUA;QPN')
(49633, "MU;W]\\]]??[S^S;HM/'W]^[:O\\SCYTGS[LVGTTF_")
(49634, 'MMF^D_AGC@\\!_:AQ)W;OQ]&P\']_?B"C_RA).P__JM.Z/^\\!')
(49635, 'MCV?JROGK\']YQ[>YX)?Z\\N[CX^X*\\B^/?S_^\'_F_,.($J:.(,!$"/]V@<')
(

(51123, 'more:')
(51124, 'SOUND.EXE')
(51125, 'selfexpanding')
(51126, 'SPEAKER.DRV,')
(51127, 'reactionary')
(51128, 'Liberal')
(51129, 'rave')
(51130, 'populist')
(51131, 'elitist')
(51132, 'incapable')
(51133, 'undoing')
(51134, 'tide')
(51135, 'Utopian')
(51136, 'arrogent')
(51137, 'jerks.')
(51138, 'disagreeing')
(51139, 'Limbaugh,')
(51140, 'conclusions.')
(51141, 'entertaining.')
(51142, "Limbaugh's")
(51143, 'oratory')
(51144, 'sizzling')
(51145, 'lacking,')
(51146, 'bankrupt')
(51147, 'concentrated')
(51148, 'movement)')
(51149, '"myopic')
(51150, 'perspective",')
(51151, 'politic')
(51152, 'Citizenry,')
(51153, 'Affairs')
(51154, 'Public.')
(51155, 'selfimportant')
(51156, 'myopic')
(51157, '"oughtabe\'s"')
(51158, 'hopelessly')
(51159, 'Hmmm,')
(51160, '"Congress')
(51161, 'thereof."')
(51162, 'Liberals)')
(51163, 'iconic')
(51164, 'others."')
(51165, 'LESS')
(51166, 'imposition')
(51167, 'specificity')
(51168, '[Analysis')
(51169, 'historical/modern')
(51170, '(how,')
(5117

In [15]:
##Creating a document term matrix for feeding to LDA model
doc_term_matrix = [dictionary.doc2bow(doc) for doc in clean_data]

In [16]:
type(doc_term_matrix),len(doc_term_matrix)

(list, 2000)

In [17]:
doc_term_matrix[1][1:10]

[(74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1)]

In [18]:
##Running the LDA Model now
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

In [19]:
# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [24]:
print(ldamodel.print_topics(num_topics=3, num_words=4))

[(0, '0.027*"I" + 0.006*"would" + 0.005*"The" + 0.004*"like"'), (1, '0.043*"." + 0.007*"The" + 0.004*"I" + 0.002*"drive"'), (2, '0.019*"I" + 0.007*"The" + 0.004*"$" + 0.004*"one"')]


In [25]:
type(ldamodel),ldamodel

(gensim.models.ldamodel.LdaModel,
 <gensim.models.ldamodel.LdaModel at 0x1a27e21b00>)

In [26]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

KeyboardInterrupt: 

In [4]:
gensim.__version__

'3.1.0'

In [99]:
import nltk

In [103]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [102]:
from nltk.corpus import gutenburg

ImportError: cannot import name 'gutenburg'

In [105]:
os.getcwd()

'/Users/rahuljain/Desktop/Python/MachineLearningwithPython/Topic_Modelling'